# Long Text Abstractive Summarization

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/long-text-abstractive](https://github.com/huseinzol05/Malaya/tree/master/example/long-text-abstractive).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

<div class="alert alert-warning">

This module trained heavily on news structure.
    
</div>

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
%%time
import malaya
from pprint import pprint

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


CPU times: user 5.06 s, sys: 727 ms, total: 5.79 s
Wall time: 5.38 s


In [2]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

### List available Transformer models

In [3]:
malaya.summarization.abstractive.available_transformer()

INFO:malaya.summarization.abstractive:tested on translated 12k CNN + DailyNews test set at https://github.com/huseinzol05/malay-dataset/tree/master/summarization


,Size (MB),Quantized Size (MB),ROUGE-1,ROUGE-2,ROUGE-L,Suggested length
t5,1250.0,481.0,0.371740,0.184714,0.258272,512.0
small-t5,355.6,195.0,0.366970,0.177330,0.254670,512.0
tiny-t5,208.0,103.0,0.302676,0.119321,0.202918,512.0
pegasus,894.0,225.0,0.251093,0.066789,0.155907,512.0
small-pegasus,293.0,74.2,0.290123,0.118788,0.192322,512.0
bigbird,910.0,230.0,0.267346,0.072391,0.161326,1536.0
small-bigbird,303.0,77.3,0.246203,0.058961,0.151590,1536.0


If you look at `Suggested length`, bigbird 3x longer than T5 and PEGASUS models.

Below is an example I combined 3 news become 1 string.

In [4]:
# https://www.astroawani.com/berita-malaysia/lagipun-kalau-isu-yang-berlaku-198788-najib-boleh-ingat-dia-nak-mandikan-keris-dengan-darah-tun-m-282301
# https://www.bharian.com.my/berita/nasional/2021/02/785400/kabinet-tidak-pernah-bangkitkan-isu-konflik-kepentingan-najib
# https://www.astroawani.com/berita-malaysia/laporan-audit-1mdb-najib-gagal-gugurkan-sri-ram-daripada-pasukan-pendakwaan-283003
# https://www.astroawani.com/berita-politik/tun-m-menyesal-lantik-tommy-thomas-sebagai-peguam-negara-281460
# https://www.astroawani.com/berita-malaysia/tun-mahathir-mahu-saya-letak-jawatan-kerana-tentangan-orang-melayu-tommy-thomas-280066

string = """
Perseteruan antara dua bekas Perdana Menteri, Tun Dr Mahathir Mohamad dan Datuk Seri Najib Tun Razak belum ada penghujungnya dengan masing-masing berbalas kenyataan di media sosial.
Selepas Najib menyanggah kenyataan tidak campur tangan dalam badan kehakiman negara dan mentertawakannya, Dr Mahathir membalasnya dengan meminta Ahli Parlimen Pekan itu memberi perhatian kepada kes 1Malaysia Development Berhad (1MDB).
Dr Mahathir juga secara sinis berkata, jika Najib boleh mengingati isu yang berlaku pada 1987 dan 1988 - isu pemilihan UMNO dan pengharaman parti itu, Najib juga boleh mengingati peristiwa dia ingin mandikan keris dengan darah.
"Saya rasa Najib tak payah campur tangan dengan tuduhan terhadap saya. Dia harus fokus kes curi duit rakyat berbilion-bilion dalam 1MDB.
"Dia juga perlu bagi perhatian saman Tommy Thomas yang kait dia dengan pembunuhan Altantuya. Lagipun, kalau isu yang berlaku 1987/88, Najib boleh ingat (peristiwa) yang dia 'hunus' keris," kata Dr Mahathir.
Sebelum ini, Najib menyanggah dakwaan Dr Mahathir yang mendakwa pembatalan pendaftaran UMNO pada 1998 sebagai bukti tidak campur tangan dalam badan kehakiman negara.
Menyokong hujahnya, Najib berkongsi apa yang berlaku pada tahun tersebut hingga menyebabkan UMNO diharamkan dan tertubuhnya UMNO baharu.

Mahkamah Tinggi di sini, hari ini, diberitahu Kabinet tidak pernah membangkitkan isu mengenai konflik kepentingan Datuk Seri Najib Razak dalam 1Malaysia Development Berhad (1MDB).

Menurut Kod Etika Bagi Anggota Pentadbiran dan Ahli Parlimen adalah menjadi amalan ahli Kabinet untuk mengisytiharkan konflik kepentingan sekiranya mempunyai pembabitan di dalam hal yang dibincangkan dalam Mesyuarat Jemaah Menteri.

Perkara itu dimaklumkan bekas Timbalan Ketua Setiausaha (Kabinet) Bahagian Kabinet Perlembagaan dan Perhubungan Jabatan Perdana Menteri (JPM), Tan Sri Mazidah Abdul Majid, dalam keterangannya pada perbicaraan kes 1MDB yang dihadapi Najib.

Kod etika itu antara lain turut menyatakan bahawa ahli Kabinet yang mempunyai kepentingan peribadi dan bercanggah dengan kepentingan kerajaan, atau membabitkan ahli keluarga, perlu meninggalkan mesyuarat dan merekodkan ketidakhadiran mereka.

Di dalam 1MDB, Najib memegang tiga jawatan iaitu Perdana Menteri, Menteri Kewangan dan Pengerusi Pengerusi Badan Penasihat 1MDB

Menjawab soalan peguam Tan Sri Muhammad Shafee Abdullah, sama ada terdapat ahli Kabinet yang membangkitkan isu bahawa Menteri Kewangan tidak seharusnya membabitkan diri dalam perbincangan itu kerana konflik kepentingan, Maziah menjawab: "Tiada."

Muhammad Shafee: Ada sesiapa yang membangkitkan perkara berhubung hal 1MBD dengan Najib?

Maziah: Tidak

Muhammad Shafee: Timbalan Perdana Menteri (ketika itu) adalah Tan Sri Muhyiddin Yassin, manakala Datuk Seri Ahmad Husni Hanadzlah adalah bekas Menteri Kewangan II. Mereka juga tidak pernah membangkitkan hal konflik kepentingan?

Mazidah: Ya, tidak pernah.

Sementara itu, menjawab soalan Timbalan Pendakwa Raya, Ahmad Akram Gharib sama ada beliau mengetahui bahawa Najib mempunyai kepentingan peribadi dalam 1MDB, Mazidah berkata: "Tidak"

Ahmad Akram: Adakah anda mengetahui bahawa Najib secara peribadi menerima duit daripada 1MDB?

Maziah: Tidak

Ahmad Akram: Sekiranya Najib secara peribadi menerima wang daripada 1MDB adakah itu konflik kepentingan dan melanggar Kod Etika Bagi Anggota Pentadbiran dan Ahli Parlimen.

Maziah: Menurut pandangan peribadi saya, ya, namun kerana ia membabitkan menteri dan perdana menteri, saya cadangkan untuk dapatkan pandangan Peguam Negara.

Terdahulu, di awal prosiding, Maziah turut memberitahu mahkamah bahawa Najib tidak pernah menyebut nama ahli perniagaan dalam buruan, Low Taek Jho atau Jho Low pada mesyuarat Kabinet sebagai individu yang membantu beliau mendapatkan sumbangan daripada kerabat diraja Arab Saudi.

"Sekiranya perkara itu dimaklumkan kepada Kabinet, maka, ia akan dicatat dalam minit mesyuarat," katanya.

Tambah Maziah, beliau hanya mendengar dan mengetahui mengenai nama Jho Low selepas timbul isu membabitkan 1MDB.

Najib, 68, menghadapi empat pertuduhan menggunakan kedudukannya untuk memperoleh suapan berjumlah RM2.3 bilion daripada dana 1MDB dan 21 pertuduhan pengubahan wang haram membabitkan jumlah wang yang sama.

Perbicaraan di hadapan Hakim Collin Lawrence Sequerah bersambung Isnin ini.

Datuk Seri Najib Tun Razak hari ini gagal dalam satu lagi cubaannya untuk menggugurkan bekas Hakim Mahkamah Persekutuan Datuk Seri Gopal Sri Ram daripada mengetuai pasukan pendakwaan dalam kes meminda laporan audit 1Malaysia Development Berhad (1MDB) melibatkan bekas perdana menteri itu.
Ini merupakan cubaan kali ketiga Najib untuk menggugurkan Sri Ram sebagai pendakwa dalam kes jenayah berkaitan 1MDB itu. Sebelum ini, satu permohonan difailkan dalam satu lagi kes 1MDB di hadapan hakim berbeza dan cubaan kedua menerusi prosiding sivil.
Ketika menolak permohonan Ahli Parlimen Pekan itu, Hakim Mahkamah Tinggi Mohamed Zaini Mazlan berkata dakwaan Najib bahawa Sri Ram terlibat dalam siasatan terhadapnya sebagai tidak ada merit.
“Tidak ada bukti kukuh untuk menyokong dakwaan pemohon dan kekal sebagai hipotesis semata-mata. Isu ini telah disiasat oleh pihak pendakwaan selaku responden semasa permohonan pemohon (Najib) yang terdahulu.
“Isu ini sudah dibincangkan dan diputuskan. Keputusan oleh mahkamah lain sebelum ini kekal dan tidak boleh diterbalikkan,” kata hakim.
Mohamed Zaini ketika memberi alasan penolakan berkata pemohon, antara lain, menjadikan komunikasi di antara bekas Peguam Negara Tan Sri Mohamed Apandi Ali dan Sri Ram sebagai bukti berat sebelah Sri Ram terhadap pemohon dan kebimbangan pemohon berhubung perkara itu adalah tidak berasas.
“Seperti juga individu lain, Sri Ram berhak mempunyai pandangan peribadi. Itu sahaja. Namun, pertimbangan berbeza akan dibuat jika beliau menunjukkan sikap berat sebelah semasa melaksanakan tugas sebagai pendakwa raya kanan. Pandangan peribadi beliau tidak boleh dianggap akan menghalang tanggungjawab beliau sebagai pendakwa raya kanan.
“Tambahan pula, kejadian itu, seperti yang dikemukakan oleh responden, berlaku ketika sebelum pelantikan Sri Ram sebagai pendakwa raya kanan. Turut penting ialah pemohon tidak membuat sebarang aduan mengenai tindak-tanduk Sri Ram ketika menjalankan perbicaraan melibatkan pemohon. Ini mengukuhkan hujah responden bahawa Sri Ram bersikap terbuka semasa menjalankan tugas sebagai pendakwa raya kanan,” kata hakim.
Mohamed Zaini seterusnya berkata pada akhirnya, mahkamah bertanggungjawab memastikan sesebuah perbicaraan dilaksanakan secara adil demi mendapatkan keadilan.
“Mahkamah akan membantu mana-mana pihak yang dilayan secara tidak adil, jika perkara tersebut berlaku. Sehubunggan itu, permohonan pemohon ditolak,” katanya.
Perbicaraan kes laporan audit 1MDB itu akan bersambung pada 22 Feb ini.
Pada prosiding hari ini, Timbalan Pendakwa Raya Ahmad Akram Gharib bertindak bagi pihak pendakwaan, manakala Najib diwakili peguam Nur Syahirah Hanapiah.
Najib, 67, dan bekas Ketua Pegawai Eksekutif 1MDB Arul Kanda Kandasamy, 45, dibicarakan atas tuduhan meminda laporan audit 1MDB.
Ahli Parlimen Pekan itu dituduh menggunakan kedudukannya untuk mengarahkan pindaan ke atas laporan audit akhir 1MDB sebelum dibentangkan kepada Jawatankuasa Kira-Kira Wang Negara bagi mengelakkan sebarang tindakan diambil terhadapnya, sementara Arul Kanda didakwa bersubahat dengan Najib dalam membuat pindaan ke atas laporan tersebut bagi melindungi Najib daripada dikenakan tindakan.
"""

string = cleaning(string)

In [5]:
len(string.split())

1020

### Load Transformer model

```python
def transformer(model: str = 'small-t5', quantized: bool = False, **kwargs):
    """
    Load Malaya transformer encoder-decoder model to generate a summary given a string.

    Parameters
    ----------
    model : str, optional (default='t2t')
        Model architecture supported. Allowed values:

        * ``'t5'`` - T5 BASE parameters.
        * ``'small-t5'`` - T5 SMALL parameters.
        * ``'tiny-t5'`` - T5 TINY parameters.
        * ``'pegasus'`` - Pegasus BASE parameters.
        * ``'small-pegasus'`` - Pegasus SMALL parameters.
        * ``'bigbird'`` - BigBird + Pegasus BASE parameters.
        * ``'small-bigbird'`` - BigBird + Pegasus SMALL parameters.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: model
        List of model classes:

        * if `t5` in model, will return `malaya.model.t5.Summarization`.
        * if `bigbird` in model, will return `malaya.model.bigbird.Summarization`.
        * if `pegasus` in model, will return `malaya.model.pegasus.Summarization`.
    """
```

In [4]:
bigbird = malaya.summarization.abstractive.transformer(model = 'bigbird')
bigbird_small = malaya.summarization.abstractive.transformer(model = 'small-bigbird')

INFO:root:running abstractive-summarization-v2/bigbird using device /device:CPU:0
INFO:root:running abstractive-summarization-v2/small-bigbird using device /device:CPU:0


#### Predict using greedy decoder

```python
def greedy_decoder(
    self,
    strings: List[str],
    postprocess: bool = False,
    **kwargs,
):
    """
    Summarize strings using greedy decoder.

    Parameters
    ----------
    strings: List[str]
    postprocess: bool, optional (default=Falsse)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: List[str]
    """
```

In [8]:
pprint(t5.greedy_decoder([string], mode = 'ringkasan'))

['Perdana Menteri, Dr Mahathir membalas dengan meminta Najib untuk memberi '
 'perhatian kepada kes 1MDB. Mahkamah Tinggi diberitahu Kabinet tidak pernah '
 'membangkitkan isu konflik kepentingan Najib. Hakim menolak permintaan Najib '
 'untuk menggugurkan Hakim Mahkamah Persekutuan Gopal Sri Ram. Ini adalah '
 'percubaan ketiga Najib untuk menggugurkan Sri Ram sebagai pendakwa dalam kes '
 '1MDB.']


In [7]:
pprint(bigbird.greedy_decoder([string]))

['Bekas Perdana Menteri , Datuk Seri Najib Razak tidak akan memberi keterangan '
 'di hadapan mahkamah berhubung kes berkenaan . Seorang lagi responden '
 'menyatakan jika perbicaraan kes terhadap Najib Razak telah diadakan , beliau '
 'akan didakwa secara peribadi . Pendakwa raya menyatakan Najib Razak tidak '
 'mempunyai bukti seberat sebelah dalam kes tersebut . Bekas Perdana Menteri , '
 'Najib Razak tidak mempunyai pandangan mengenai perkara ini .']


In [8]:
pprint(bigbird_small.greedy_decoder([string]))

['BARU : Seri Najib berkata , beliau tidak pernah secara peribadi meminta '
 'untuk menggugurkan nama Serie - A dalam kes tersebut . Dia berkata bahawa '
 'dia tidak pernah secara peribadi menyentuh perkara itu sebelum ini . Russell '
 'Maziah berkata , beliau tidak pernah secara peribadi meminta untuk '
 'menggugurkan nama Serie A . S .']


#### Predict using nucleus decoder

```python
def nucleus_decoder(
    self,
    strings: List[str],
    top_p: float = 0.7,
    temperature: float = 0.2,
    postprocess: bool = False,
    **kwargs,
):
    """
    Summarize strings using nucleus decoder.

    Parameters
    ----------
    strings: List[str]
    top_p: float, (default=0.7)
        cumulative distribution and cut off as soon as the CDF exceeds `top_p`.
    temperature: float, (default=0.3)
        logits * -log(random.uniform) * temperature.
    postprocess: bool, optional (default=False)
        If True, will filter sentence generated using ROUGE score and removed international news publisher.

    Returns
    -------
    result: List[str]
    """
```

In [9]:
pprint(bigbird.nucleus_decoder([string]))

['Bekas Perdana Menteri , Datuk Seri Najib Razak tidak akan memberi keterangan '
 'di hadapan mahkamah berhubung kes berkenaan . Seorang lagi responden '
 'menyatakan sama ada Najib Razak menyertai parti politik yang dikaitkan '
 'dengan pembunuhan tokoh - tokoh penting negara . Pendakwa raya menyatakan '
 'Peguam Negara tidak akan membuat pindaan terhadap kes yang sedang dihadapi .']


In [10]:
pprint(bigbird_small.nucleus_decoder([string]))

['BARU : Seri Najib berkata , beliau masih belum mendedahkan pandangan ahli '
 'Kabinet . Mahathir Mohamad awas dakwaan konflik interestsome tidak pernah '
 'dibincangkan sebelum ini . Beliau berkata , pegawai istana yang dinaikinya '
 'itu berkata , mereka tidak pernah secara peribadi merupakan perkara yang '
 'betul untuk dilakukan .']
